# ISI Datamart Demonstration
---
This demonstration illustrates the following capabilities:

- Entity linking to  [Wikidata](http://wikidata.org), [Wikidata statistics](https://www.wikidata.org/wiki/Special:Statistics), [statistics graphs](https://grafana.wikimedia.org/d/000000175/wikidata-datamodel-statements?refresh=30m&orgId=1), [most viewed pages](https://tools.wmflabs.org/topviews/?project=wikidata.org&platform=all-access&date=last-year&excludes=)
- Augmentation with data from the Wikidata knowledge graph
- Augmentation with data from Excel, CSV and other structured sources
- Enriching Wikidata 

In [1]:
import sys, os
# sys.stdout = open(os.devnull, 'w')
from wikifier import utils
from datamart.entries_new import D3MDatamart, D3MJoinSpec
import pandas as pd
d3mDatamart = D3MDatamart()
# this is our original input dataset
inputs_ds_loc = "/Users/pszekely/Downloads/datamart_demo/DA_poverty_estimation/TRAIN/dataset_TRAIN/datasetDoc.json"
# pd.set_option('display.max_columns', None)
original_dataset = utils.load_d3m_dataset(inputs_ds_loc)


Using TensorFlow backend.
/Users/pszekely/anaconda/envs/datamart/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Our Original Dataset
---
We start with a dataset that has the number of people in poverty in different counties in the United States. In this demo we are using data from Florida and Georgia only (so it runs faster).

In [2]:
#original_dataset['learningData'].head()
original_dataset['learningData'].sample(n=15)

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016
122,1715,12017,FL,Citrus County,4,23472
15,329,13197,GA,Marion County,2,1975
53,1570,13309,GA,Wheeler County,9,2111
75,2445,13005,GA,Bacon County,7,2524
21,505,13255,GA,Spalding County,1,14243
123,1724,12051,FL,Hendry County,4,9712
79,2591,13027,GA,Brooks County,3,3854
102,94,12059,FL,Holmes County,6,4276
13,273,13125,GA,Glascock County,9,539
32,791,13277,GA,Tift County,4,8669


## Linking To Wikidata
---
Wikidata contains over 80 million identifiers for entities. Datamart can scan a dataset, automatically identify columns containing entity identifiers, and link the identifiers to the appropriate entity in Wikidata.

Here is the wikified data. Clicking on the links takes you to the corresponding wikidata pages where you can see all the data available for each entity.

In [3]:
wikified_dataset = utils.wikifier_for_d3m_all(input_ds=original_dataset).value
# wikified_dataset['learningData']
utils.pretty_print(wikified_dataset,"wikifier")

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016,FIPS_wikidata,State_wikidata
0,1,13297,GA,Walton County,1,11385,Q498312,Q58428702
1,2,13137,GA,Habersham County,6,6500,Q501096,Q58428702
2,6,13059,GA,Clarke County,3,31950,Q112061,Q58428702
3,36,13055,GA,Chattooga County,6,4716,Q486179,Q58428702
4,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702
5,60,13105,GA,Elbert County,6,4197,Q492016,Q58428702
6,82,13195,GA,Madison County,3,4255,Q156387,Q58428702
7,92,13263,GA,Talbot County,8,1447,Q498356,Q58428702
8,116,13211,GA,Morgan County,6,2358,Q493083,Q58428702
9,143,13165,GA,Jenkins County,6,2606,Q389551,Q58428702


## Searching Datamart Using Our Wikified Data
---
Datamart finds multiple datasets that can be used to augment the poverty dataset. The results show the title, the columns available in each dataset, and the columns that will be used to join the candidate dataset to the poverty dataset.

In [4]:
search_results = d3mDatamart.search_with_data(supplied_data=wikified_dataset)
# wiki_search_results.display()
utils.print_search_results(search_results)

,title,columns,join columns
0,wikidata search result for FIPS_wikidata,"population, area, inception, violent crime off...",FIPS_wikidata
1,wikidata search result for State_wikidata,publication date,State_wikidata
2,Unemployment and median household income for t...,"FIPStxt, State, Area_name, Rural_urban_continu...",[Area]
3,Unemployment and median household income for t...,"FIPStxt, State, Area_name, Rural_urban_continu...",[Area]
4,"Poverty estimates for the U.S., States, and co...","FIPStxt, State, Area_Name, Rural-urban_Continu...",[Area]
5,"Poverty estimates for the U.S., States, and co...","FIPStxt, State, Area_Name, Rural-urban_Continu...",[Area]
6,Educational attainment for adults age 25 and o...,"FIPS Code, State, Area name, 2003 Rural-urban ...",[Area]
7,Educational attainment for adults age 25 and o...,"FIPS Code, State, Area name, 2003 Rural-urban ...",[Area]
8,PopulationEstimates with q nodes,"FIPS, State, Area_Name, Rural-urban_Continuum ...",[Area]
9,PopulationEstimates with q nodes,"FIPS, State, Area_Name, Rural-urban_Continuum ...",[Area]


## Using Search Results To Augment Your Data
---
The first search result, from Wikidata, augments our data using population, area and inception date of counties.

In [5]:
wiki_search_result = search_results[0]
augmented_dataset = wiki_search_result.augment(supplied_data=wikified_dataset)
utils.pretty_print(augmented_dataset,"wiki_augment")

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016,FIPS_wikidata,State_wikidata,Aggravated assault,Burglary,Larceny-theft,Motor vehicle theft,Property crime,Robbery,area,inception,murder and non-negligent manslaughter,population,violent crime offenses
0,1,13297,GA,Walton County,1,11385,Q498312,Q58428702,55,171,542,82,795,10,nan,1818-01-01T00:00:00Z,0,85754,73
1,2,13137,GA,Habersham County,6,6500,Q501096,Q58428702,nan,nan,nan,nan,nan,nan,723,1818-12-15T00:00:00Z,nan,43300,nan
2,6,13059,GA,Clarke County,3,31950,Q112061,Q58428702,0,0,0,0,0,0,314,1801-01-01T00:00:00Z,0,121265,0
3,36,13055,GA,Chattooga County,6,4716,Q486179,Q58428702,21,101,233,12,346,3,812,1838-01-01T00:00:00Z,0,25138,26
4,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,8,31,935,39,461,881,1832-12-02T00:00:00Z,16,717190,1262
5,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,2184,8490,0,39,461,881,1832-12-02T00:00:00Z,16,717190,79
6,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,66,8,31,935,11609,461,881,1832-12-02T00:00:00Z,16,717190,79
7,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,66,2184,31,0,11609,461,881,1832-12-02T00:00:00Z,0,717190,79
8,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,66,2184,8490,935,11609,8,881,1832-12-02T00:00:00Z,0,717190,79
9,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,8,8490,0,11609,461,881,1832-12-02T00:00:00Z,16,717190,1262


## Repeat Augmentation With Additional Search Results From Wikidata
---
The second search result, also from Wikidata, augments our using data from states. This adds columns with information about the states.

In [6]:
augmented_dataset = search_results[1].augment(supplied_data=augmented_dataset)
utils.pretty_print(augmented_dataset,"wiki_augment")

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016,FIPS_wikidata,State_wikidata,Aggravated assault,Burglary,Larceny-theft,Motor vehicle theft,Property crime,Robbery,area,inception,murder and non-negligent manslaughter,population,violent crime offenses
0,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,153,0,0,0,0,36,1516,1777-02-05T00:00:00Z,2,81508,0
1,825,13151,GA,Henry County,1,21101,Q492053,Q58428702,168,994,2697,445,4136,0,840,1821-05-15T00:00:00Z,0,211128,0
2,182,13115,GA,Floyd County,3,14596,Q486389,Q58428702,11,21,754,4,81,11,1343,1832-12-03T00:00:00Z,1,95821,84
3,646,13285,GA,Troup County,4,13731,Q498295,Q58428702,nan,nan,nan,nan,nan,nan,1155,1825-06-08T00:00:00Z,nan,69053,nan
4,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,66,2184,31,0,11609,461,881,1832-12-02T00:00:00Z,16,717190,1262
5,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,8,31,935,39,8,881,1832-12-02T00:00:00Z,16,717190,79
6,182,13115,GA,Floyd County,3,14596,Q486389,Q58428702,55,351,754,4,1216,11,1343,1832-12-03T00:00:00Z,1,95821,16
7,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,0,0,0,114,1852,0,1516,1777-02-05T00:00:00Z,0,81508,201
8,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,8,8490,935,11609,461,881,1832-12-02T00:00:00Z,0,717190,79
9,825,13151,GA,Henry County,1,21101,Q492053,Q58428702,168,0,135,445,135,120,840,1821-05-15T00:00:00Z,0,211128,0



## Download Data
---
Some of the search results are datasets indexed from the web, they are present in the search results because they can be joined with our wikified dataset.

Let's inspect a few of these datasets.

__Download the search result contains poverty information.__

Datamart computes a foreign key to join the downloaded dataset with the supplied data (last column)

In [7]:
downloaded_dataset = search_results[12].download(supplied_data=wikified_dataset)
# utils.pretty_print(downloaded_dataset)
utils.pretty_print(downloaded_dataset,"download")

,FIPStxt,State,Area_Name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2017,CI90LBAll_2017,CI90UBALL_2017,PCTPOVALL_2017,CI90LBALLP_2017,CI90UBALLP_2017,POV017_2017,CI90LB017_2017,CI90UB017_2017,PCTPOV017_2017,CI90LB017P_2017,CI90UB017P_2017,POV517_2017,CI90LB517_2017,CI90UB517_2017,PCTPOV517_2017,CI90LB517P_2017,CI90UB517P_2017,MEDHHINC_2017,CI90LBINC_2017,CI90UBINC_2017,POV04_2017,CI90LB04_2017,CI90UB04_2017,PCTPOV04_2017,CI90LB04P_2017,CI90UB04P_2017,FIPStxt_wikidata,joining_pairs
391,12121,FL,Suwannee County,6,6,6,6,"8,299","6,497","10,101",20.3,15.9,24.7,"2,728","2,061","3,395",30,22.7,37.3,"1,884","1,384","2,384",28.2,20.7,35.7,"44,144","41,156","47,132",nan,nan,nan,nan,nan,nan,Q501036,[132]
361,12063,FL,Jackson County,6,6,6,6,"7,264","5,523","9,005",18,13.7,22.3,"2,117","1,528","2,706",24,17.3,30.7,"1,565","1,118","2,012",24.5,17.5,31.5,"41,524","37,686","45,362",nan,nan,nan,nan,nan,nan,Q488537,[131]
378,12095,FL,Orange County,1,1,1,1,"201,528","188,575","214,481",15.3,14.3,16.3,"65,440","58,650","72,230",21.9,19.6,24.2,"44,755","39,282","50,228",20.8,18.3,23.3,"54,021","52,629","55,413",nan,nan,nan,nan,nan,nan,Q488543,[130]
371,12083,FL,Marion County,2,2,2,2,"55,880","49,007","62,753",16.2,14.2,18.2,"17,133","13,930","20,336",26.3,21.4,31.2,"12,253","9,813","14,693",25.8,20.7,30.9,"43,772","42,127","45,417",nan,nan,nan,nan,nan,nan,Q501014,[129]
357,12055,FL,Highlands County,4,5,3,2,"20,051","17,160","22,942",19.8,16.9,22.7,"5,964","4,855","7,073",34,27.7,40.3,"4,240","3,391","5,089",33.1,26.5,39.7,"37,445","34,115","40,775",nan,nan,nan,nan,nan,nan,Q488885,[128]
343,12027,FL,DeSoto County,6,5,6,5,"8,766","7,195","10,337",26.1,21.4,30.8,"2,505","2,009","3,001",37.2,29.8,44.6,"1,801","1,429","2,173",36.7,29.1,44.3,"37,342","33,967","40,717",nan,nan,nan,nan,nan,nan,Q488796,[127]
345,12031,FL,Duval County,1,1,1,1,"138,069","128,011","148,127",15.1,14,16.2,"48,079","42,653","53,505",23,20.4,25.6,"31,702","27,402","36,002",21.7,18.8,24.6,"52,105","50,735","53,475",nan,nan,nan,nan,nan,nan,Q493605,[126]
350,12041,FL,Gilchrist County,3,2,2,2,"2,675","2,067","3,283",16.1,12.4,19.8,871,650,"1,092",24.4,18.2,30.6,632,464,800,24.8,18.2,31.4,"42,880","38,671","47,089",nan,nan,nan,nan,nan,nan,Q111720,[125]
379,12097,FL,Osceola County,1,1,1,1,"48,892","41,590","56,194",14,11.9,16.1,"18,351","14,938","21,764",21.4,17.4,25.4,"12,571","9,944","15,198",19.8,15.7,23.9,"49,284","46,191","52,377",nan,nan,nan,nan,nan,nan,Q501067,[124]
355,12051,FL,Hendry County,4,5,4,3,"9,525","7,776","11,274",23.9,19.5,28.3,"3,741","2,934","4,548",34.7,27.2,42.2,"2,662","2,085","3,239",34.8,27.3,42.3,"38,361","34,378","42,344",nan,nan,nan,nan,nan,nan,Q488488,[123]


__Download the search result contains unemployment information.__

In [8]:
downloaded_dataset = search_results[13].download(supplied_data=wikified_dataset)
# utils.pretty_print(downloaded_dataset)
utils.pretty_print(downloaded_dataset,"download")

,FIPStxt,State,Area_Name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2017,CI90LBAll_2017,CI90UBALL_2017,PCTPOVALL_2017,CI90LBALLP_2017,CI90UBALLP_2017,POV017_2017,CI90LB017_2017,CI90UB017_2017,PCTPOV017_2017,CI90LB017P_2017,CI90UB017P_2017,POV517_2017,CI90LB517_2017,CI90UB517_2017,PCTPOV517_2017,CI90LB517P_2017,CI90UB517P_2017,MEDHHINC_2017,CI90LBINC_2017,CI90UBINC_2017,POV04_2017,CI90LB04_2017,CI90UB04_2017,PCTPOV04_2017,CI90LB04P_2017,CI90UB04P_2017,FIPStxt_wikidata,joining_pairs
391,12121,FL,Suwannee County,6,6,6,6,"8,299","6,497","10,101",20.3,15.9,24.7,"2,728","2,061","3,395",30,22.7,37.3,"1,884","1,384","2,384",28.2,20.7,35.7,"44,144","41,156","47,132",nan,nan,nan,nan,nan,nan,Q501036,[132]
361,12063,FL,Jackson County,6,6,6,6,"7,264","5,523","9,005",18,13.7,22.3,"2,117","1,528","2,706",24,17.3,30.7,"1,565","1,118","2,012",24.5,17.5,31.5,"41,524","37,686","45,362",nan,nan,nan,nan,nan,nan,Q488537,[131]
378,12095,FL,Orange County,1,1,1,1,"201,528","188,575","214,481",15.3,14.3,16.3,"65,440","58,650","72,230",21.9,19.6,24.2,"44,755","39,282","50,228",20.8,18.3,23.3,"54,021","52,629","55,413",nan,nan,nan,nan,nan,nan,Q488543,[130]
371,12083,FL,Marion County,2,2,2,2,"55,880","49,007","62,753",16.2,14.2,18.2,"17,133","13,930","20,336",26.3,21.4,31.2,"12,253","9,813","14,693",25.8,20.7,30.9,"43,772","42,127","45,417",nan,nan,nan,nan,nan,nan,Q501014,[129]
357,12055,FL,Highlands County,4,5,3,2,"20,051","17,160","22,942",19.8,16.9,22.7,"5,964","4,855","7,073",34,27.7,40.3,"4,240","3,391","5,089",33.1,26.5,39.7,"37,445","34,115","40,775",nan,nan,nan,nan,nan,nan,Q488885,[128]
343,12027,FL,DeSoto County,6,5,6,5,"8,766","7,195","10,337",26.1,21.4,30.8,"2,505","2,009","3,001",37.2,29.8,44.6,"1,801","1,429","2,173",36.7,29.1,44.3,"37,342","33,967","40,717",nan,nan,nan,nan,nan,nan,Q488796,[127]
345,12031,FL,Duval County,1,1,1,1,"138,069","128,011","148,127",15.1,14,16.2,"48,079","42,653","53,505",23,20.4,25.6,"31,702","27,402","36,002",21.7,18.8,24.6,"52,105","50,735","53,475",nan,nan,nan,nan,nan,nan,Q493605,[126]
350,12041,FL,Gilchrist County,3,2,2,2,"2,675","2,067","3,283",16.1,12.4,19.8,871,650,"1,092",24.4,18.2,30.6,632,464,800,24.8,18.2,31.4,"42,880","38,671","47,089",nan,nan,nan,nan,nan,nan,Q111720,[125]
379,12097,FL,Osceola County,1,1,1,1,"48,892","41,590","56,194",14,11.9,16.1,"18,351","14,938","21,764",21.4,17.4,25.4,"12,571","9,944","15,198",19.8,15.7,23.9,"49,284","46,191","52,377",nan,nan,nan,nan,nan,nan,Q501067,[124]
355,12051,FL,Hendry County,4,5,4,3,"9,525","7,776","11,274",23.9,19.5,28.3,"3,741","2,934","4,548",34.7,27.2,42.2,"2,662","2,085","3,239",34.8,27.3,42.3,"38,361","34,378","42,344",nan,nan,nan,nan,nan,nan,Q488488,[123]


## Augment With Datasets From The Web
---
Datasets from the web can also be used to augment our original data.

Let's augment using the poverty data as it is useful to predict the number of people in poverty. Many new columns appear at the end.

In [9]:
augmented_dataset = search_results[12].augment(supplied_data=augmented_dataset)
utils.pretty_print(augmented_dataset,"wiki_augment")

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016,FIPS_wikidata,State_wikidata,Aggravated assault,Burglary,Larceny-theft,Motor vehicle theft,Property crime,Robbery,area,inception,murder and non-negligent manslaughter,population,violent crime offenses,Area_Name,CI90LB017P_2017,CI90LB017_2017,CI90LB04P_2017,CI90LB04_2017,CI90LB517P_2017,CI90LB517_2017,CI90LBALLP_2017,CI90LBAll_2017,CI90LBINC_2017,CI90UB017P_2017,CI90UB017_2017,CI90UB04P_2017,CI90UB04_2017,CI90UB517P_2017,CI90UB517_2017,CI90UBALLP_2017,CI90UBALL_2017,CI90UBINC_2017,FIPStxt,FIPStxt_wikidata,MEDHHINC_2017,PCTPOV017_2017,PCTPOV04_2017,PCTPOV517_2017,PCTPOVALL_2017,POV017_2017,POV04_2017,POV517_2017,POVALL_2017,Rural-urban_Continuum_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013
0,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,153,422,0,0,0,36,1516,1777-02-05T00:00:00Z,0,81508,0,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
1,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,0,0,0,114,1852,36,1516,1777-02-05T00:00:00Z,0,81508,201,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
2,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,153,0,0,114,0,36,1516,1777-02-05T00:00:00Z,2,81508,201,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
3,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,153,422,1316,0,0,0,1516,1777-02-05T00:00:00Z,0,81508,0,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
4,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,0,0,0,114,1852,36,1516,1777-02-05T00:00:00Z,2,81508,0,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
5,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,0,0,0,0,1852,36,1516,1777-02-05T00:00:00Z,0,81508,201,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
6,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,153,0,1316,0,0,36,1516,1777-02-05T00:00:00Z,2,81508,0,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
7,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,0,0,0,0,0,0,1516,1777-02-05T00:00:00Z,0,81508,0,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
8,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,0,0,0,114,0,36,1516,1777-02-05T00:00:00Z,2,81508,0,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2
9,1319,13127,GA,Glynn County,3,15916,Q487016,Q58428702,0,422,1316,0,0,36,1516,1777-02-05T00:00:00Z,0,81508,201,Glynn County,23.5,"4,359",nan,nan,20.8,"2,838",14,"11,776","47,392",35.3,"6,565",nan,nan,32.8,"4,472",20,"16,832","54,088",13127,Q487016,"50,740",29.4,nan,26.8,17,"5,462",nan,"3,655","14,304",3,3,2,2


## Discovering And Using More Data
---
Crime data may be useful to predict poverty, but no crime data is currently available in Datamart.

Searching in Google for `fbi crime statistics by county`  produces this search result:

[<img src="images/google-search-fbi.png" alt="Google Search Result" title="Google Search Result" /> ](https://ucr.fbi.gov/crime-in-the-u.s)

After navigating to this page, click on `2016`, then `Crime in the U.S. 2016`, then `Violent Crime`. You can explore the various crime datasets. Let's choose `Table 8`, which has crime data for all states, broken doown by county. For example, the Georgia [page](https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-8/table-8-state-cuts/georgia.xls) contains crime data for counties in Georgia.

This crime data can be downloaded in Excel using the `Download Excel` [link](https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-8/table-8-state-cuts/georgia.xls/output.xls).

<img src="images/fbi-crime-data-georgia.png" alt="Georgia Crime Data" title="Georgia Crime Data" />

---

Challenges for using this data:
- The data for each state is in a separate file
- The column headers start in row 6
- The spreadsheet has notes at the end, and the notes start in different rows for different states
- The name of the state and the year are in the metadata rows (rows 2 and 4)

## Augmenting Wikidata With Data Extracted From Tables
--- 
After the table understanding step, the data can be indexed in Datamart and used for augmentation. The challenge with the FBI crime data is that the data for each state is in a separate file. Augmentation of our original dataset requires combining the data from multiple files.

Datamart addresses this challenge by mapping the table data to Wikidata and uploading the data to Datamart's Wikidata clone where it can be queried regardless of the file where it came from.

### Download The FBI Crime Data

In [10]:
utils.download_FBI_data(["Georgia", "Florida"])
# Without parameters it downloads the data for all the states
# utils.download_FBI_data()

### Use DIG To Map The Spreadsheets To Wikidata
A DIG script converts the spreadsheet data to Wikidata using a simple API for augmenting Wikidata.

In [11]:
utils.generate_FBI_data(["Georgia", "Florida"])
# utils.generate_FBI_data()

### Upload The RDF Triples To Wikidata

In [12]:
utils.upload_FBI_data(["Georgia", "Florida"])
# utils.upload_FBI_data()

### View Of Datamart Additions To Wikidata
Wikidata provides a [user interface](https://test.wikidata.org/wiki/Special:NewProperty) to define properties.

We defined [properties to represent crime data](http://tinyurl.com/y5g7juu6).

The uploaded FBI data can be visualized, taking advantage of latitude/longitude coordinates present in Wikidata:

- [Map ](http://tinyurl.com/y2a7b7a2) of crime data by county, colored by file where data was present
- [Map ](http://tinyurl.com/y2k66mcd) of crime data by county colored by severity 
- [Map ](http://tinyurl.com/yxwh24vr) of crime data by county colored by severity, per 100,000 inhabitants 


## Search Datamart Again
---
The FBI data is now available as new columns to augment the poverty data.

In [13]:
search_results = d3mDatamart.search_with_data(supplied_data=wikified_dataset)
utils.print_search_results(search_results)

,title,columns,join columns
0,wikidata search result for FIPS_wikidata,"population, area, inception, violent crime off...",FIPS_wikidata
1,wikidata search result for State_wikidata,publication date,State_wikidata
2,Unemployment and median household income for t...,"FIPStxt, State, Area_name, Rural_urban_continu...",[Area]
3,Unemployment and median household income for t...,"FIPStxt, State, Area_name, Rural_urban_continu...",[Area]
4,"Poverty estimates for the U.S., States, and co...","FIPStxt, State, Area_Name, Rural-urban_Continu...",[Area]
5,"Poverty estimates for the U.S., States, and co...","FIPStxt, State, Area_Name, Rural-urban_Continu...",[Area]
6,Educational attainment for adults age 25 and o...,"FIPS Code, State, Area name, 2003 Rural-urban ...",[Area]
7,Educational attainment for adults age 25 and o...,"FIPS Code, State, Area name, 2003 Rural-urban ...",[Area]
8,PopulationEstimates with q nodes,"FIPS, State, Area_Name, Rural-urban_Continuum ...",[Area]
9,PopulationEstimates with q nodes,"FIPS, State, Area_Name, Rural-urban_Continuum ...",[Area]


## Augment Using The FBI Data
The first search result has the FBI data.

In [14]:
wiki_search_result = search_results[0]
fbi_augmented_dataset = wiki_search_result.augment(supplied_data=wikified_dataset)
utils.pretty_print(fbi_augmented_dataset,"wiki_augment")

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016,FIPS_wikidata,State_wikidata,Aggravated assault,Burglary,Larceny-theft,Motor vehicle theft,Property crime,Robbery,area,inception,murder and non-negligent manslaughter,population,violent crime offenses
0,1,13297,GA,Walton County,1,11385,Q498312,Q58428702,55,171,542,82,795,10,nan,1818-01-01T00:00:00Z,0,85754,73
1,2,13137,GA,Habersham County,6,6500,Q501096,Q58428702,nan,nan,nan,nan,nan,nan,723,1818-12-15T00:00:00Z,nan,43300,nan
2,6,13059,GA,Clarke County,3,31950,Q112061,Q58428702,0,0,0,0,0,0,314,1801-01-01T00:00:00Z,0,121265,0
3,36,13055,GA,Chattooga County,6,4716,Q486179,Q58428702,21,101,233,12,346,3,812,1838-01-01T00:00:00Z,0,25138,26
4,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,8,8490,935,39,461,881,1832-12-02T00:00:00Z,0,717190,79
5,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,8,8490,935,39,461,881,1832-12-02T00:00:00Z,0,717190,1262
6,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,66,2184,8490,935,11609,461,881,1832-12-02T00:00:00Z,16,717190,1262
7,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,66,2184,8490,0,11609,461,881,1832-12-02T00:00:00Z,0,717190,79
8,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,8,31,935,11609,461,881,1832-12-02T00:00:00Z,0,717190,1262
9,46,13067,GA,Cobb County,1,73446,Q484247,Q58428702,697,2184,31,935,11609,8,881,1832-12-02T00:00:00Z,0,717190,79


__Remove the FBI data from the Datamart Wikidata installation__

In [15]:
# utils.clean_FBI_data()